In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score


print(sklearn.__version__)


1.5.2


In [17]:
df = pd.read_csv(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\human-resources-analytics\HR_comma_sep.csv")
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [18]:
X = df.drop(columns=['left'])
y = df['left'].astype('category')


In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=[
        'object',
        'category',
    ])),
    (ohe, make_column_selector(dtype_include=[
        'object',
        'category',
    ])),
    verbose_feature_names_out=False,
).set_output(transform='pandas')

lr = LogisticRegression(random_state=24)

kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

pipe = Pipeline([
    ("ct", ct), 
    ("lr", lr)
])

params = {
    "lr__solver": [
        "lbfgs",
        "liblinear",
        "newton-cg",
        "sag",
        "saga"
        "newton-cholesky"
    ],
    "lr__multi_class": [
        "ovr",
        "multinomial"
    ],
    "lr__C" : np.linspace(0.001, 100, 20),
}

gcv = GridSearchCV(
    estimator=pipe,
    param_grid=params,
    cv=kfold,
)

gcv.fit(X, y)

print(gcv.best_params_)
print(gcv.best_score_)


pd_cv = pd.DataFrame(gcv.cv_results_)
print(pd_cv.shape)  

{'lr__C': 10.527210526315788, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs'}
0.8026675558519505
(200, 16)


In [25]:
X_test = pd.read_csv(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\human-resources-analytics\tst_hr.csv")

In [26]:
X_test.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Department,salary
0,0.11,0.88,7,272,4,0,0,sales,medium
1,0.72,0.87,5,223,5,0,0,sales,low
2,0.37,0.52,2,159,3,0,0,sales,low
3,0.41,0.50,2,153,3,0,0,sales,low
4,0.67,0.61,3,202,2,0,0,technical,medium


In [32]:
best_lr = gcv.best_estimator_

y_pred = best_lr.predict(X_test)
y_pred_proba = best_lr.predict_proba(X_test)
print(y_pred)
print(pd.DataFrame(y_pred_proba).idxmax(axis=1).values)

[1 0 1 1 0 0 1 0 0 0 0 0 0 0 0]
[1 0 1 1 0 0 1 0 0 0 0 0 0 0 0]
